In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from matplotlib.ticker import AutoMinorLocator

Industry_Portfolios_Daily = pd.read_csv(r'48_Industry_Portfolios_Daily.csv')
Industry_Portfolios_Daily.replace(-99.99,np.nan, inplace=True)
Industry_Portfolios_Daily = Industry_Portfolios_Daily.dropna(how='any')
Industry_Portfolios_Daily

,Unnamed: 0,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
11903,19690701,0.00,0.62,0.43,0.54,0.18,0.78,0.84,-0.01,0.58,...,1.19,0.66,0.66,0.38,1.22,0.26,0.50,0.99,0.60,1.79
11904,19690702,0.76,1.11,0.27,1.29,0.16,2.05,1.43,1.59,1.32,...,1.85,0.81,1.65,0.84,1.39,1.51,1.77,0.94,1.19,1.04
11905,19690703,0.31,1.23,0.50,2.02,1.59,1.40,0.83,0.79,1.03,...,1.73,0.87,1.14,0.79,1.57,2.18,0.46,2.13,1.23,3.06
11906,19690707,-0.04,-0.59,-0.41,0.11,-0.02,-0.43,-1.51,-0.04,-0.53,...,-0.82,-0.98,-0.60,-0.20,-1.34,-0.25,-1.07,-1.62,-0.61,-1.68
11907,19690708,0.46,-0.89,-0.79,-1.19,-1.09,-1.66,-1.68,-0.55,-1.46,...,-1.25,-1.25,-1.63,-1.22,-1.82,-1.17,-1.72,-2.13,-1.55,-2.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25015,20210624,2.08,0.74,0.18,1.36,1.59,-0.07,0.67,1.21,0.89,...,0.38,0.77,1.31,0.84,0.96,1.22,0.91,1.52,1.49,1.15
25016,20210625,-1.24,0.87,0.59,0.82,-0.80,0.43,0.58,0.76,1.70,...,0.73,-0.67,-0.48,0.65,0.13,0.30,1.15,-0.72,0.55,0.28
25017,20210628,-0.72,-0.02,-0.81,0.03,0.27,0.39,-1.89,-0.94,-0.44,...,-0.24,-1.67,-0.58,-0.53,-0.86,-1.26,-0.95,-0.88,-1.02,-0.68
25018,20210629,-0.42,-0.26,-0.61,0.00,-0.32,0.10,-0.74,-0.19,-0.43,...,-0.55,-0.24,2.33,-0.53,-0.03,-0.54,-0.10,-0.80,0.06,-1.11


In [2]:
scaler = MinMaxScaler()
timestamp = 60
future_window_size = 12
train_size=1500
test_size=600 # select a test size so that: (test_size % future_window_size = 0) for "cleaner" predictions

if((train_size+test_size)>Industry_Portfolios_Daily.shape[0]):
    print("ERROR: train_size + test_size must be lower than",Industry_Portfolios_Daily.shape[0])
else:
    #pick the latest data
    df=Industry_Portfolios_Daily.iloc[-(train_size+test_size):,1:]
    data = df.to_numpy()
    print("Data shape:",data.shape,"\n")
    print("Data:\n",data)

Data shape: (2100, 48) 

Data:
 [[-0.2   0.18 -1.02 ...  2.76 -0.66  0.63]
 [-0.81  1.01  0.31 ... -0.85 -0.02  0.14]
 [ 1.05  0.17  0.06 ...  1.92  0.28  0.36]
 ...
 [-0.72 -0.02 -0.81 ... -0.88 -1.02 -0.68]
 [-0.42 -0.26 -0.61 ... -0.8   0.06 -1.11]
 [-0.07  0.43 -0.17 ... -0.04  0.37  0.16]]


In [3]:
results = pd.DataFrame()

for stocknum in range(0,data.shape[1]):
    print("Predicting values of stock \"",Industry_Portfolios_Daily.columns[stocknum + 1],"\" (",stocknum+1,"/",data.shape[1],") ...")
#Prepare train data
    train_data = data[0:train_size]
    current_stock_train = train_data[:,stocknum]
    #from [1,2,3,4,5] to [[1],[2],[3],[4],[5]]
    current_stock_train = current_stock_train.reshape(-1,1)
    #create stock prices arrays
    current_stock_train[0] = current_stock_train[0]/100 + 1;
    for i in range (1,train_size):
        current_stock_train[i] = current_stock_train[i]/100 + current_stock_train[i-1];
    current_stock_train_last_value = current_stock_train[train_size-1]

#Scale data
    scaler.fit(current_stock_train[0:train_size,:])
    current_stock_train_scaled = current_stock_train.copy()
    current_stock_train_scaled[0:train_size,:] = scaler.transform(current_stock_train_scaled[0:train_size,:])

#Creating a data structure with 60 timesteps and 1 output
    X_train_list = []
    Y_train_list = []
    for i in range(timestamp, train_size):
        X_train_list.append(current_stock_train_scaled[i-timestamp:i, 0])
        Y_train_list.append(current_stock_train_scaled[i, 0])
    X_train, Y_train = np.array(X_train_list), np.array(Y_train_list)

#Reshaping
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

#Initialising the RNN
    regressor = Sequential()
    # Adding the first LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))
    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    # Adding a third LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    # Adding a fourth LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    # Adding the output layer
    regressor.add(Dense(units = 1))
    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    # Fitting the RNN to the Training set
    regressor.fit(X_train, Y_train, epochs = 25, batch_size = 2)


#Prepare test data
    data = df.to_numpy()
    test_data = data[train_size:train_size + test_size].copy()
    current_stock_test = test_data[:,stocknum].copy()
    #from [1,2,3,4,5] to [[1],[2],[3],[4],[5]]
    current_stock_test = current_stock_test.reshape(-1,1)
    #create stock prices arrays
    current_stock_test[0] = current_stock_train_last_value + current_stock_test[0]/100;
    for i in range (1,test_size):
        current_stock_test[i] = current_stock_test[i]/100 + current_stock_test[i-1];
#Scale test data
    current_stock_test_scaled = current_stock_test.copy()
    scaler.fit(current_stock_test_scaled[0:test_size,:])
    current_stock_test_scaled[0:test_size,:] = scaler.transform(current_stock_test_scaled[0:test_size,:])
#Creating a data structure with 60 timesteps and 1 output, and predicting the future window
    predicted_prices_list = []
    for x in range(timestamp,test_size,future_window_size):
        for i in range (x,x+future_window_size):
            X_test = current_stock_test_scaled[i-timestamp:i, 0].copy()
            #use earlier predictions to predict further in
            for j in range(0,i%future_window_size):
                X_test[-(j+1)] = predicted_prices_list[-(j+1)]
    #         print(X_test[-future_window_size:])
            X_test = np.reshape(X_test, (1, X_test.shape[0], 1))
            predicted_stock_price = regressor.predict(X_test)
    #         print("predicted: ",predicted_stock_price)
            predicted_prices_list.append(predicted_stock_price)
    for x in range(len(predicted_prices_list)):
            predicted_prices_list[x] = scaler.inverse_transform(predicted_prices_list[x])
    predicted_prices = np.array(predicted_prices_list)
    results[Industry_Portfolios_Daily.columns[stocknum + 1]] = predicted_prices.tolist()
    print(results.iloc[:,-3:])
    print("Corelation:",np.average(np.corrcoef(current_stock_test[timestamp:,0], predicted_prices[:,0,0])),"\n")

Predicting values of stock " Agric " ( 0 / 48 ) ...
Epoch 1/25
720/720 [==============================] - 24s 28ms/step - loss: 0.0135
Epoch 2/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0077
Epoch 3/25
720/720 [==============================] - 20s 27ms/step - loss: 0.0056
Epoch 4/25
720/720 [==============================] - 20s 27ms/step - loss: 0.0050
Epoch 5/25
720/720 [==============================] - 20s 27ms/step - loss: 0.0040
Epoch 6/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0034
Epoch 7/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0032
Epoch 8/25
720/720 [==============================] - 20s 27ms/step - loss: 0.0026
Epoch 9/25
720/720 [==============================] - 20s 27ms/step - loss: 0.0024
Epoch 10/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0023
Epoch 11/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0021
Epoch 12/25
720/720 [============

Epoch 1/25
720/720 [==============================] - 23s 28ms/step - loss: 0.0110
Epoch 2/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0055
Epoch 3/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0041
Epoch 4/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0035
Epoch 5/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0030
Epoch 6/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0028
Epoch 7/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0025
Epoch 8/25
720/720 [==============================] - 20s 28ms/step - loss: 0.0023
Epoch 9/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0020 0s - loss: 
Epoch 10/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0020
Epoch 11/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0019
Epoch 12/25
720/720 [==============================] - 21s 29ms/step - lo

720/720 [==============================] - 22s 30ms/step - loss: 0.0039
Epoch 7/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0034
Epoch 8/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0031 0s - l
Epoch 9/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0028
Epoch 10/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0028
Epoch 11/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0028
Epoch 12/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0024
Epoch 13/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0023 1s - loss:  - ETA: 0s
Epoch 14/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0022
Epoch 15/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0023
Epoch 16/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0022
Epoch 17/25
720/720 [==============================] - 22s 30m

Epoch 1/25
720/720 [==============================] - 25s 30ms/step - loss: 0.0106
Epoch 2/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0049
Epoch 3/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0041
Epoch 4/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0033
Epoch 5/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0030
Epoch 6/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0025 2
Epoch 7/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0021
Epoch 8/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0020
Epoch 9/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0019 
Epoch 10/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0016
Epoch 11/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0017
Epoch 12/25
720/720 [==============================] - 21s 30ms/step - loss: 0.001

720/720 [==============================] - 22s 30ms/step - loss: 0.0027 0s -
Epoch 8/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0022
Epoch 9/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0020
Epoch 10/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0022
Epoch 11/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0017
Epoch 12/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0018ETA: 3s
Epoch 13/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0015
Epoch 14/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0017 1s - - ET
Epoch 15/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0015
Epoch 16/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0016
Epoch 17/25
720/720 [==============================] - 22s 30ms/step - loss: 0.0014
Epoch 18/25
720/720 [==============================] - 22s 30ms/step

...

720/720 [==============================] - 21s 29ms/step - loss: 0.0026
Epoch 8/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0024
Epoch 9/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0021
Epoch 10/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0021
Epoch 11/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0018
Epoch 12/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0020
Epoch 13/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0019 1s -  - ETA: 0s 
Epoch 14/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0018
Epoch 15/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0018
Epoch 16/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0016
Epoch 17/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0015
Epoch 18/25
720/720 [==============================] - 21s 29ms/step - lo

Epoch 1/25
720/720 [==============================] - 26s 32ms/step - loss: 0.0093
Epoch 2/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0044
Epoch 3/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0047
Epoch 4/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0032
Epoch 5/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0028
Epoch 6/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0022
Epoch 7/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0024
Epoch 8/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0021
Epoch 9/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0019
Epoch 10/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0016
Epoch 11/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0018
Epoch 12/25
720/720 [==============================] - 23s 32ms/step - loss: 0.0015
E

720/720 [==============================] - 21s 29ms/step - loss: 0.0024
Epoch 8/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0022
Epoch 9/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0022
Epoch 10/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0019
Epoch 11/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0017
Epoch 12/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0017
Epoch 13/25
720/720 [==============================] - 21s 30ms/step - loss: 0.0017
Epoch 14/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0016
Epoch 15/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0015
Epoch 16/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0015
Epoch 17/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0015 0s - loss: 0.00
Epoch 18/25
720/720 [==============================] - 21s 29ms/step - los

Epoch 1/25
720/720 [==============================] - 24s 29ms/step - loss: 0.0100
Epoch 2/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0053
Epoch 3/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0038
Epoch 4/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0031
Epoch 5/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0031
Epoch 6/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0030
Epoch 7/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0022
Epoch 8/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0021
Epoch 9/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0021
Epoch 10/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0021
Epoch 11/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0019
Epoch 12/25
720/720 [==============================] - 21s 29ms/step - loss: 0.0019
E

'Agric,Food ,Soda ,Beer ,Smoke,Toys ,Fun  ,Books,Hshld,Clths,Hlth ,MedEq,Drugs,Chems,Rubbr,Txtls,BldMt,Cnstr,Steel,FabPr,Mach ,ElcEq,Autos,Aero ,Ships,Guns ,Gold ,Mines,Coal ,Oil  ,Util ,Telcm,PerSv,BusSv,Comps,Chips,LabEq,Paper,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin  ,Other\r\n[[1.2022215127944946]],[[1.8942153453826904]],[[2.6435606479644775]],[[2.370537519454956]],[[2.4801881313323975]],[[1.8497133255004883]],[[2.1563973426818848]],[[1.8398616313934326]],[[1.5378713607788086]],[[1.9889822006225586]],[[1.89354407787323]],[[1.5873762369155884]],[[2.0329413414001465]],[[1.5128140449523926]],[[1.498212218284607]],[[1.591914415359497]],[[1.8351006507873535]],[[1.5449144840240479]],[[1.4936753511428833]],[[1.505976676940918]],[[1.7230600118637085]],[[1.7225205898284912]],[[1.7239493131637573]],[[1.9012548923492432]],[[2.0734527111053467]],[[2.293631076812744]],[[0.5006551742553711]],[[1.0157380104064941]],[[-1.0653218030929565]],[[0.4517529010772705]],[[1.836525797843933]],[[

In [1]:
results.to_csv("LSTM_Predictions",index=False)

SyntaxError: invalid syntax (<ipython-input-1-89bc36800748>, line 1)